In this lecture we're going to talk about querying DataFrames. The first step in the process is to understand boolean masking. Boolean masking is the heart of fast and efficient querying in numpy and pandas, and its analogous to bit masking used in other areas of computational science. By the end of this lecture we'll understand how Boolean masking works, and how to apply this to a DataFrame to get out data we're interested in.

A Boolean mask is an array which can be of one dimension like a Series, or two dimensions like a DataFrame, where each of the values in the array are either true or false. This array is essentially overlaid on top of the data structure that we're querying. And any cell alligned with the true value will be admitted into our final result, and any cell aligned with a false value will not.

In [2]:
# lets start with an example and import our graduate admission dataset. first we'll bring in pandas
import pandas as pd
#then we'll load in our CSV file
df = pd.read_csv('/content/Admission_Predict.csv', index_col=0)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
# we'll clean up a couple of poorly named columns like we did in a previous lecture
df.columns = [x.lower().strip() for x in df.columns]
df.head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [4]:
# boolean masks are created by applying operators directly to the pandas Series or DataFrame objects.
# for instance, in our graduate admission dataset, we might be interested in seeing only those students
# that have a chance higher than 0.7

# to build a Boolean mask for this query, we want to project the chance of admit columns using the
# indexing operator and apply the greater that operator with a comparison value of 0.7. this is
# essentially broadcasting a comparison operator, greater that, with the results being returned as
# a Boolean Series. The resultant Series is indexed where the value of each call is either True or False
# depending on whether a student has a chance of admin higher than 0.7

admit_mask = df['chance of admit'] > 0.7
admit_mask

Serial No.
1       True
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398     True
399    False
400     True
Name: chance of admit, Length: 400, dtype: bool

In [5]:
# this is pretty fundamental, so take a moment to look at this. the result of broadcasting a comparison
# operator is a Boolean mask - true or false values depending upon the results of the compariosn. underneath
# pandas is applying the comparison operator you specified through vectorization (so efficiently and in
# parralel) to all of the values in the array you specified which, in this case, is the chance of admit column
# of the dataframe. the result is a series, since only one column is being operator on, filled with
# either True or False values, which is what the comparison operator returnrs

In [6]:
# so, what do we do with the boolean mask once we have formed it? well, we can just lay it on top of the
# data to "hide" the data we dont want, which is represented by all of the False values. we do this by using
# the .where() function on the original DataFrame.
df.where(admit_mask).head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# we see that the resulting data frame keeps the original indexed values, and only data which met
# the condition was retained. All of the rows which didnt meet the condtion have NaN data instead,
# but these rows were not dropped from our dataset.

# the next step is, if we dont want the NaN data, we use the dropna() function
df.where(admit_mask).dropna().head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90


In [8]:
# the returned DataFrame now has all of the NaN rows dropped. notice the index now includes
# one through four and six, but not five.

# despite being really handy, where() isnt actually used that often. instead, the pandas devs
# created a shorthand syntax which combines where() and dropna(), doing both at one. and, in
# typical fashion, the just overloaded the indexing operator to do this!

df[df['chance of admit'] > 0.7].head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90


In [9]:
# this is relatively much harder to read, but it's also very much more common when we're reading other
# people's code, so it's important to be able to understand it. just reviewing this indexing operator on
# DataFrame, it now does two things:

# it can be called with a string parameter to project a single column
df["gre score"].head()

Serial No.
1    337
2    324
3    316
4    322
5    314
Name: gre score, dtype: int64

In [10]:
# or we can send it a list of columns as strings
df[["gre score", "toefl score"]].head()

,gre score,toefl score
Serial No.,,
1,337,118
2,324,107
3,316,104
4,322,110
5,314,103


In [11]:
# or we can send it a boolean mask
df[df['gre score'] > 320].head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90
7,321,109,3,3.0,4.0,8.20,1,0.75


In [12]:
# and each of these is mimicing functionality from either .loc() or .where().dropna().

In [13]:
# before we leave this, lets talk about combining multiple boolean masks, such as multiple criteria for
# including. in bitmasking in other places in computer science this is done with "and", if both masks must be
# true for a True value to be in the final mask, or "or" if only one needs to be True.

# unfortunately, it doesnt feel quite as natural in pandas. for instance, if we want to take two boolean
# series and "and" them together
(df['chance of admit'] > 0.7) and (df['chance of admit'] < 0.9)

ValueError: ignored

In [14]:
# this doesnt work. the problem is that we have series objects, and python underneath doesnt know how to
# compare two series using "and" or "or". instead, the pandas authors have overwritten the pipe | and ampersant &
# operators to handle this for us

In [15]:
(df['chance of admit'] > 0.7) & (df['chance of admit'] < 0.9)

Serial No.
1      False
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

In [16]:
# the problem is that python is trying to bitwise "and" a 0.7 and a pandas dataframe, when you
# really want to bitwise and the broadcasted dataframes together

In [17]:
# another way to do this is to just get rid of the comparison operator completely, and instread
# use the built in functions which mimic this approach
df['chance of admit'].gt(0.7) & df['chance of admit'].lt(0.9)

Serial No.
1      False
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

In [21]:
# these functions are build right into the Series and DataFrame objects, so you can chain them
# too, which results in the same answer and  the use of no visual operators. you can decide what
# looks best for you
#df['chance of admit'].gt(0.7).lt(0.9)

In this lecture, we have learned to query dataframe using boolean masking, which is extremely importnat and often used in the world of data science. with boolean masking, we can select data based on the criteria we desire and, frankly, we'll use it everywhhere. we've also seen how there are many different ways to query the DataFrame.